# Multilevel Multigrid Example

In [48]:
# dependencies
using LFAToolkit
using LinearAlgebra
using Pkg
Pkg.activate("./")
Pkg.instantiate()
using Plots

 Activating environment at `~/Dev/LFAToolkit.jl/examples/jupyter/Project.toml`


In [49]:
# setup
finep = 9
midp5 = 5
midp3 = 3
coarsep = 2
dimension = 2

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

mtofbasis5 = TensorH1LagrangeBasis(midp5, finep, dimension, lagrangequadrature=true)
mtofbasis3 = TensorH1LagrangeBasis(midp3, midp5, dimension, lagrangequadrature=true)
ctombasis = TensorH1LagrangeBasis(coarsep, midp3, dimension, lagrangequadrature=true)

# diffusion operators
finediffusion = GalleryOperator("diffusion", finep, finep, mesh)
middiffusion5 = GalleryOperator("diffusion", midp5, finep, mesh)
middiffusion3 = GalleryOperator("diffusion", midp3, finep, mesh)
coarsediffusion = GalleryOperator("diffusion", coarsep, finep, mesh)

# Jacobi smoother
finejacobi = Jacobi(finediffusion)
midjacobi5 = Jacobi(middiffusion5)
midjacobi3 = Jacobi(middiffusion3)

# p-multigrid preconditioner
midmultigrid3 = PMultigrid(middiffusion3, coarsediffusion, midjacobi3, [ctombasis])
midmultigrid5 = PMultigrid(middiffusion5, midmultigrid3, midjacobi5, [mtofbasis3])
multigrid = PMultigrid(finediffusion, midmultigrid5, finejacobi, [mtofbasis5])

p-multigrid preconditioner

In [ ]:
# full operator symbols
numberruns = 100
numberruns_w = 100
maxeigenvalue = 0
θ_min = -π/2
θ_max = 3π/2
ω_max = 1.0

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # setup
    smoothingfactor = zeros(numberruns)
    ω_min = [0.0, 1.0]
    v = [2, 2]

    # compute
    for i in 1:numberruns
        ω = [ω_max*i/numberruns]
        for j in 1:numberruns
            θ = [θ_min + (θ_max - θ_min)*j/numberruns]
            if abs(θ[1] % 2π) >  π/512
                A = computesymbols(multigrid, ω, v, θ)
                eigenvalues = [abs(val) for val in eigvals(A)]
                smoothingfactor[i] = max(smoothingfactor[i], eigenvalues...)
            end
        end
        if smoothingfactor[i] < ω_min[2]
            ω_min = [ω[1], smoothingfactor[i]]
        end
    end

    # plot
    println("Min ω: ", ω_min)
    xrange = 0:ω_max/(numberruns-1):ω_max
    plot(xrange,
        smoothingfactor,
        xlabel="ω",
        xtickfont=font(12, "Courier"),
        ylabel="spectral radius",
        ytickfont=font(12, "Courier"),
        linewidth=3,
        legend=:none,
        title="Two-Grid Convergence Factor"
    )
    ylims!(0.0, max(smoothingfactor...))
    savefig("two_grid_converge_5_to_3")
# -- 2D --
elseif dimension == 2
    # setup
    smoothingfactor = zeros(numberruns_w)
    ω_min = [0.0, 1.0]
    v = [2, 2]

    # compute
    for i in 1:numberruns_w
        ω = [ω_max*i/numberruns]
        for j in 1:numberruns, k in 1:numberruns
            θ = [
                θ_min + (θ_max - θ_min)*j/numberruns,
                θ_min + (θ_max - θ_min)*k/numberruns
            ]
            if sqrt(abs(θ[1] % 2π)^2 + abs(θ[2] % 2π)^2) > π/128
                A = computesymbols(multigrid, ω, v, θ)
                eigenvalues = [abs(val) for val in eigvals(A)]
                smoothingfactor[i] = max(smoothingfactor[i], eigenvalues...)
            end
        end
        if smoothingfactor[i] < ω_min[2]
            ω_min = [ω[1], smoothingfactor[i]]
        end
    end

    # plot
    println("Min ω: ", ω_min)
    xrange = 0:ω_max/(numberruns_w-1):ω_max
    plot(xrange,
        smoothingfactor,
        xlabel="ω",
        xtickfont=font(12, "Courier"),
        ylabel="spectral radius",
        ytickfont=font(12, "Courier"),
        linewidth=3,
        legend=:none,
        title="Two-Grid Convergence Factor"
    )
    ylims!(0.0, max(1.0, max(smoothingfactor...)))
    #savefig("two_grid_converge_5_to_3_2d")
end

In [ ]:
Min ω: [0.776, 0.28000973345867536]
Min ω: [0.786, 0.27497519366350426]
Min ω: [0.790, 0.27298331271746656]
Min ω: [0.791, 0.2724872903974415]
Min ω: [0.793, 0.2714975774808143]
Min ω: [0.794, 0.2710038852549067]
Min ω: [0.795, 0.2705109681074074]
Min ω: [0.796, 0.27367459612685263]
Min ω: [0.806, 0.3230906415884941]